#### Install dependencies

In [21]:
!pip install snscrape
!pip install textblob
!pip install pandas
!pip install vaderSentiment
!pip install tqdm

^C
ERROR: Operation cancelled by user


----
#### Needed imports

In [2]:
import pandas as pd
import snscrape.modules.twitter as sntwitter
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from tqdm import tnrange, tqdm_notebook, tqdm

----
#### Start Mining Tweets

In [9]:
query = "(crash OR crashing OR cair OR queda OR subir OR subida OR bullish OR bearish) (#bitcoin OR #ethereum OR #crypto OR #BTC OR #ETH OR #bullmarket OR #bearmarket OR #bearish OR #bullish) until:2023-03-01 since:2022-04-03"
tweets = []
limit = 1000

#TODO: meter aqui a barra de progresso ( https://github.com/tqdm/tqdm )

for tweet in sntwitter.TwitterHashtagScraper(query).get_items():
    
    if len(tweets) == limit:
        break
    else:
        tweets.append([tweet.date, tweet.url, tweet.user.username, tweet.sourceLabel, tweet.user.location, tweet.content, tweet.likeCount, tweet.retweetCount,  tweet.quoteCount, tweet.replyCount])
        
df = pd.DataFrame(tweets, columns=['Date', 'TweetURL','User', 'Source', 'Location', 'Tweet', 'Likes_Count','Retweet_Count', 'Quote_Count', 'Reply_Count'])

df.to_csv('../data/bullishTweets.csv')

0it [00:00, ?it/s]/tmp/ipykernel_46322/2228346395.py:12: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  tweets.append([tweet.date, tweet.url, tweet.user.username, tweet.sourceLabel, tweet.user.location, tweet.content, tweet.likeCount, tweet.retweetCount,  tweet.quoteCount, tweet.replyCount])
1000it [00:55, 17.99it/s]


#### Sentiment Analysis with VADER

In [8]:
analyzer = SentimentIntensityAnalyzer()
compound = []
for i,s in enumerate(tqdm(df['Tweet'])):
    vs = analyzer.polarity_scores(s)
    compound.append(vs["compound"])
df["compoundVader"] = compound
df.head(2)

df.to_csv('../data/compoundAnalysis.csv')

100%|██████████| 1000/1000 [00:00<00:00, 6464.84it/s]


#### Sentiment Analysis with TextBlob

In [7]:
compound = []
for i,s in enumerate(tqdm(df['Tweet'])):
    vs = TextBlob(s).sentiment
    compound.append(vs)
df["compoundTextBlob"] = compound
df.head(2)

df.to_csv('../data/compoundAnalysis.csv')

100%|██████████| 1000/1000 [00:00<00:00, 1789.69it/s]
